In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import Huber
import tensorflow as tf

/tmp/ipykernel_55030/2096283702.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
2024-04-28 03:26:27.037104: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-28 03:26:27.057743: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-28 03:26

In [2]:
df = pd.read_csv('./input/copper_returns_5d_final.csv')
df.describe()

,LMCADS03,LMCADY,DXY,SPX,BCOM,MXWD,XAU,XAG,LMCADY_acu_5d,LMCADY_std_5d
count,5539.000000,5539.000000,5539.000000,5539.000000,5539.000000,5539.000000,5539.000000,5539.000000,5539.000000,5539.000000
mean,-0.000185,-0.000178,0.000007,-0.000252,0.000072,-0.000205,-0.000274,-0.000106,-0.000867,0.014218
std,0.016059,0.016464,0.004793,0.011794,0.010250,0.009880,0.010722,0.019587,0.034782,0.008721
min,-0.112019,-0.110645,-0.024921,-0.103782,-0.054910,-0.085172,-0.097378,-0.123485,-0.185825,0.000805
25%,-0.008649,-0.008880,-0.002635,-0.005469,-0.005554,-0.004878,-0.005978,-0.009770,-0.021102,0.008616
50%,0.000000,0.000000,0.000000,-0.000390,0.000000,-0.000643,-0.000497,-0.000771,-0.001844,0.012148
75%,0.007562,0.007760,0.002658,0.003949,0.005375,0.003807,0.004908,0.008074,0.017065,0.017520
max,0.109603,0.109134,0.027541,0.136158,0.066117,0.105134,0.099792,0.226116,0.259832,0.091981


In [3]:
features = df.drop(['Date', 'LMCADY_std_5d', 'LMCADY_acu_5d'], axis=1)

# scaler = MinMaxScaler()
# scaled_features = scaler.fit_transform(df.drop(['Date','LMCADY_std_5d','LMCADY_acu_5d'], axis=1))
# features = scaled_features

print(features.shape)


(5539, 8)


In [4]:
def crearSecuencias(data, n_steps):
    X, y = [], []
    data = data.values  # Asegurarse de que 'data' es un array de NumPy
    for i in range(n_steps, len(data)):
        X.append(data[i-n_steps:i, :-2])  # las variables excepto los target
        y.append(data[i, -2:])            # los target
    return np.array(X), np.array(y)

In [5]:
n_steps = 25  # ventana modificable
X, y = crearSecuencias(features, n_steps)
print(X.shape, y.shape)


(5514, 25, 6) (5514, 2)


In [6]:
# verificar que haya secuencia
print(X[0])
print(X[1])

[[ 1.24125480e-03  1.04704023e-03 -7.67018217e-04  2.80768240e-03
   6.12699272e-03  1.18171426e-03]
 [-7.32559450e-04 -1.81555969e-04  2.01496834e-03  3.06428091e-03
  -5.82881005e-03  2.39912759e-03]
 [ 9.47386229e-03  9.32156546e-03 -4.02183281e-03  1.40423142e-03
   7.49108445e-03  2.58536835e-03]
 [-2.51382604e-03 -2.12952297e-03 -5.96096529e-03 -3.22615725e-03
  -1.95286967e-03 -6.74036817e-03]
 [ 5.43234767e-03  5.89306530e-03  4.15900958e-03 -8.82552224e-03
   3.44083378e-03 -6.09207516e-03]
 [ 1.25327243e-02  1.29976857e-02 -2.31169332e-03 -5.61744594e-03
   8.81608423e-04 -2.37935137e-03]
 [-1.87039278e-03 -1.95969112e-03 -1.44815601e-03 -6.27837698e-03
  -4.90577206e-03 -5.00337015e-03]
 [-2.03373016e-02 -2.06556280e-02 -6.76786232e-04  6.52519303e-03
  -1.12108641e-03  6.55272140e-03]
 [ 4.44444444e-03  5.71656278e-03 -5.51470588e-03  2.87934328e-03
  -8.75065479e-04  3.42975474e-03]
 [-3.02453232e-02 -3.08780478e-02  1.65385738e-03  1.92824826e-03
  -8.15826797e-03  5.2882

In [7]:
y

array([[-0.00355659,  0.00080418],
       [-0.00179456,  0.01726519],
       [-0.00456399, -0.0211992 ],
       ...,
       [ 0.        , -0.00153139],
       [-0.01480427, -0.01472393],
       [ 0.00577951, -0.00788709]])

In [8]:
# desviación estándar de los target
np.std(y, axis=0)

array([0.0107347 , 0.01960124])

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=0)


In [10]:
def rmse(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_true - y_pred)))

In [11]:
model = Sequential([
    Conv1D(filters=128, kernel_size=2, activation='relu', input_shape=(n_steps, X_train.shape[2])),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(50, activation='relu'),
    Dense(2)  # dos variables target
])

initial_learning_rate = 0.001
optimizer = Adam(learning_rate=initial_learning_rate)

huber_loss = Huber(delta=0.05)  # Puedes ajustar delta según sea necesario
model.compile(optimizer=optimizer, loss=huber_loss, metrics=[rmse])


2024-04-28 03:26:27.773427: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-28 03:26:27.794339: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-28 03:26:27.794463: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [12]:
early_stopping = EarlyStopping(
    monitor='rmse',  # Monitorea la pérdida de validación
    patience=15,         # Número de epochs sin mejora después del cual el entrenamiento será detenido
    restore_best_weights=True  # Restaura los pesos del modelo desde la epoch con la mejor pérdida de validación
)

history = model.fit(X_train, y_train, epochs=75, verbose=1, validation_split=0.15, callbacks=[early_stopping])


Epoch 1/75


2024-04-28 03:26:29.628432: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-04-28 03:26:30.254336: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-04-28 03:26:30.310068: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-04-28 03:26:30.945706: I external/local_xla/xla/service/service.cc:168] XLA service 0x776fa80168b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-04-28 03:26:30.945721: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3080, Compute Capability 8.6
2024-04-28 03:26:30.948978: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1714292791.002599   55157 device_compiler.

125/125 [==============================] - 3s 3ms/step - loss: 1.1803e-04 - rmse: 0.0155 - val_loss: 1.1921e-04 - val_rmse: 0.0156
Epoch 2/75
125/125 [==============================] - 0s 1ms/step - loss: 1.1569e-04 - rmse: 0.0153 - val_loss: 1.1752e-04 - val_rmse: 0.0155
Epoch 3/75
125/125 [==============================] - 0s 1ms/step - loss: 1.1456e-04 - rmse: 0.0152 - val_loss: 1.1761e-04 - val_rmse: 0.0155
Epoch 4/75
125/125 [==============================] - 0s 1ms/step - loss: 1.1457e-04 - rmse: 0.0152 - val_loss: 1.1737e-04 - val_rmse: 0.0155
Epoch 5/75
125/125 [==============================] - 0s 1ms/step - loss: 1.1306e-04 - rmse: 0.0151 - val_loss: 1.1654e-04 - val_rmse: 0.0155
Epoch 6/75
125/125 [==============================] - 0s 1ms/step - loss: 1.1128e-04 - rmse: 0.0150 - val_loss: 1.1738e-04 - val_rmse: 0.0155
Epoch 7/75
125/125 [==============================] - 0s 1ms/step - loss: 1.0926e-04 - rmse: 0.0148 - val_loss: 1.1660e-04 - val_rmse: 0.0155
Epoch 8/75
125/12

In [14]:
loss = model.evaluate(X_test, y_test)
print(f'Loss on test data: {loss}')


26/26 [==============================] - 0s 704us/step - loss: 1.6935e-04 - rmse: 0.0187
Loss on test data: [0.00016934925224632025, 0.01869456097483635]
